# miRNA-Seq

## Validation by Real-Time PCR

On previous [friday](qRT-PCR_validation.ipynb), I have tested the Female EF sample RNA50 (diluted 1/50), instead of RNA5 (diluted 1/5). Today I repeated the RT-qPCR with the correct sample.

In this experiment I also added 3 controls, to test the reaction without cDNA (using vvi-mir156 primer), without primers, and a pair of wells completelly empty.

### List of miRNAs tested
* vvi-mir156(d) (positive control, see below)
* vvi-mir160(d) (may not work, see below)
* vvi-mir168
* vvi-mir172(b)
* vii-mir828(a) (may not work, see below)

#### Notes
* vvi-mir156(d) was used as positive control, as this primer already amplified [previously](qRT-PCR_optimizations.ipynb). Also, I wanted to test the consequences of duplicating the Evagreen quantity, so this test will evaluate that.

* vvi-mir160(b) and vvi-mir828(a) may not work as these miRNAs are not expected to be present in this sample.

* Test was performed in duplicates.

* Primer R was Oligo(dT).


### Protocol

| Component | 1X (µL) | 11X (µL) |
| :-------- |-------: | -------: |
| Evagreen  |    10.0 |    110.0 |
| cDNA      |     0.5 |      5.5 |
| Primer F  |     0.5 |      --- |
| Primer R  |     0.5 |      5.5 |
| H20       |     8.5 |     93.5 |

Control wells were set directly in the well (it was not used mix)


### Plate Setup

* B2-3: vvi-mir156
* B4-5: vvi-mir172
* B6-7: (control: no primer)
* C2-3: vvi-mir160
* C4-5: vvi-mir828
* C6-7: (control: empty)
* D2-3: vvi-mir168
* D4-5: (Control: no cDNA)


### Run setup

1. 95 ºC for 30 seconds
2. 95 ºC for 5 seconds
3. 60 ºC for 15 seconds
4. Go to 2. 45 times

### Loading required modules

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# From the directory that is running notebook
path = './'

### Preparing data

In [ ]:
columns = {
    'Well/Cycles': 'cycle',
    
    'B02': 'mir156_r1', 'B03': 'mir156_r2',
    'B04': 'mir172_r1', 'B05': 'mir172_r2',
    'B06': 'no_primer_r1', 'B07': 'no_primer_r2',
        
    'C02': 'mir160_r1', 'C03': 'mir160_r2',
    'C04': 'mir828_r1', 'C05': 'mir828_r2',
    'C06': 'empty_r1', 'C07': 'empty_r2',
    
    'D02': 'mir168_r1', 'D03': 'mir168_r2',
    'D04': 'no_cdna_r1', 'D05': 'no_cdna_r2'
}

### Open amplification file

In [ ]:
amplification = pd.read_csv(f'{path}amplification_data_rfu.tsv',
                            sep = '\t',
                            header = 0,
                            index_col = 0
                           )

amplification = amplification.rename(columns = columns)
amplification.head()

### Verifing amplification curves

In [ ]:
columns_list = list(columns.values())

# The first item will be cycle, with is not relevant
columns_list.pop(0)

# Remove reaction number
columns_list = list(map(lambda x: x.rsplit('_', 1)[0], columns_list))

# Remove the duplicates
columns_list = list(set(columns_list))

# Plot the amplifications
for sample in columns_list:
    amplification[[sample + '_r1', sample + '_r2']].plot()

### Considerations on amplification

The amplifications are not exactly as expected from miRNA-Seq:
* vvi-mir156 had amplification on only one of the reactions. Both should had work (success: 1/2)
* vvi-mir160 was not expected to express, and no expression was found (success: 2/2 | 3/4)
* vvi-mir168 was expected to express, but no expression was found (success: 0/2 | 3/6)
* vvi-mir172 was expected to express and in fact, it express in one reaction (success: 1/2 | 4/8)
* vvi-mir828 is a negative control and no expression was found (success: 2/2 | 6/10)
* no_cdna did not amplify as expected (success: 2/2 | 8/12)
* no_primer we had one unexpected reaction (success: 1/2 | 9/14)
* empty did not react (success: 2/2 | 11/16)

Success: 67%

### Opening melting file

In [ ]:
melting = pd.read_csv(f'{path}melting_data_rfu.tsv',
                      sep = '\t',
                      header = 0,
                      index_col = 0
                     )

melting = melting.rename(columns = columns)
melting.head()

#### Calculating the melting derivate

The melting derivate will be calculated according to the formula:
`-delta(RFU) / delta(temperature)`

On this case, the difference in temperature is always 0.5 as that was the parameters set on the experiment.

In [ ]:
def derivate_melting(values):
    delta_value = []
    for i in range(1, len(values)):
        j = i - 1
        delta_value.append(values[i] - values[j])
    
    derivate = list(map(lambda x: -x/0.5, delta_value))
    return(derivate)

In [ ]:
for sample in columns_list:
    derivate_1 = derivate_melting(melting[sample + '_r1'].values.tolist())
    plt.plot(derivate_1)
    
    derivate_2 = derivate_melting(melting[sample + '_r2'].values.tolist())    
    plt.plot(derivate_2)
    
    plt.title(sample)
    
    plt.show()